### Library Import

In [1]:
from data.load_dataset import load_dataset
from data.merge_dataset import merge_dataset
from data.data_preprocessing import *
from data.feature_engineering import *
from model.inference import save_csv
from model.feature_select import select_features
from model.data_split import split_features_and_target
from model.model_train import set_model, optuna_train
#from model.TreeModel import XGBoost
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd
import optuna
import torch

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Load

In [2]:
# 기존 데이터 불러오기
train_data, test_data, sample_submission, interest_data, subway_data, school_data, park_data = load_dataset()
# 기존 데이터에 새로운 feature들을 병합한 데이터프레임 불러오기
train_data, test_data = merge_dataset(train_data, test_data, interest_data, subway_data, school_data, park_data)

### Data Preprocessing

In [3]:
# 위치 중복도 낮은 행 삭제
train_data = delete_low_density(train_data, 2, 6)

# built_year가 2024인 행 삭제
train_data = train_data[train_data["built_year"] < 2024]
train_data.reset_index(drop=True, inplace=True)

### Feature Engineering

#### Average Deposit by Region

In [4]:
# 클러스터별(region) 평균 전세가 생성
region_mean_prices = train_data.groupby("region")["deposit"].mean().reset_index()
region_mean_prices.columns = ["region", "mean_deposit"]
region_mean_prices["mean_deposit_category"] = region_mean_prices["mean_deposit"] // 10000

# train_data와 test_data에 region_mean_prices 병합 (test에는 train의 평균가격이 병합된다.)
train_data = train_data.merge(region_mean_prices, on="region", how="left")
test_data = test_data.merge(region_mean_prices, on="region", how="left")

#### Log Transformation

- `deposit`
- `area_m2`
- `nearest_subway_distance`
- `nearest_school_distance`
- `nearest_park_distance`
- `nearest_leader_distance`

In [5]:
train_data, test_data = apply_log_transformation(train_data, test_data)

/opt/conda/lib/python3.10/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: divide by zero encountered in log1p
  result = func(self.values, **kwargs)
/opt/conda/lib/python3.10/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)
/opt/conda/lib/python3.10/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: divide by zero encountered in log1p
  result = func(self.values, **kwargs)
/opt/conda/lib/python3.10/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)


### Train Data Split

In [9]:
# train data split
X, y = split_features_and_target(train_data)

# 1. 원래 데이터 쓰는 사람용 (불필요한 변수 drop: 로그변환한 변수)
X.drop(columns=["index", "log_area_m2", "log_subway_distance", "log_school_distance", "log_park_distance", "log_leader_distance"], inplace=True)
y.drop(columns="log_deposit", inplace=True)

# 2. 로그변환한 데이터 쓰는 사람용 (불필요한 변수 drop: 로그변환하기 전 변수)
#X.drop(columns=["index", "area_m2", "nearest_subway_distance", "nearest_school_distance", "nearest_park_distance", "nearest_leader_distance"], inplace=True)
#y.drop(columns="deposit", inplace=True)

# 영균 컬럼
# selected_cols = [
#    "log_area_m2", "built_year", "latitude", "longitude", "log_leader_distance", "log_subway_distance", "log_park_distance", "contract_year_month", "num_of_subways_within_radius", "park_exists", "region"
# ]
# X = X[selected_cols]
# y.drop(columns="deposit", inplace=True)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
X_train = X_train.values
X_valid = X_valid.values
y_train = y_train.values
y_valid = y_valid.values

/tmp/ipykernel_3422478/3176361316.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.drop(columns="log_deposit", inplace=True)


### Model Train

#### Vanilla TabNet

In [10]:
# def train_model(model, X: pd.DataFrame, y: pd.DataFrame) -> float:
#     """모델을 학습하고 검증 MAE를 계산하는 함수입니다.

#     Args:
#         model: 수행하려는 모델
#         X (pd.DataFrame): 독립 변수
#         y (pd.DataFrame): 예측 변수. deposit과 log_deposit 열로 나뉨.

#     Returns:
#         float: 검증 MAE
#     """
#     X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
#     X_train = X_train.values
#     X_valid = X_valid.values
#     y_train = y_train.values
#     y_valid = y_valid.values

#     # 모델 학습
#     model.fit(X_train, y_train, 
#           eval_set=[(X_train, y_train),(X_valid, y_valid)], 
#           eval_name=["train", "valid"],
#           eval_metric=["mae"],
#           loss_fn=torch.nn.L1Loss(),
#           max_epochs=30, 
#           patience=10,
#           batch_size=8192,
#           drop_last=False,
#           warm_start=True  # warm start 활성화
#     )
#     print("모델 학습이 완료됐습니다. ⏲")

#     # 2. 로그변환한 데이터 쓰는 사람용
#     y_train = np.expm1(y_train) # -> 로그변환 변수 사용시 활성화
#     y_valid = np.expm1(y_valid) # -> 로그변환 변수 사용시 활성화

#     # 예측 및 로그 변환 복구
#     y_train_pred = model.predict(X_train)
#     y_train_pred = np.expm1(y_train_pred) # 2. 로그변환한 데이터 쓰는 사람용 -> log_deposit의 inverse log 처리
#     y_valid_pred = model.predict(X_valid)
#     y_valid_pred = np.expm1(y_valid_pred) # 2. 로그변환한 데이터 쓰는 사람용 -> log_deposit의 inverse log 처리

#     # 학습 MAE, 검증 MAE 계산
#     mae_train = mean_absolute_error(y_train, y_train_pred)
#     mae_valid = mean_absolute_error(y_valid, y_valid_pred)
#     print("학습 결과..! 🎉")
#     print(f"Train MAE: {mae_train:.4f}, Valid MAE: {mae_valid:.4f}")

#     return mae_train, mae_valid


# def objective(trial):
#     """
# 	    Optuna를 이용하여 Hyperparameter 튜닝을 수행하는 함수입니다.
# 	  """
#     # n_d를 먼저 제안합니다.
#     n_d = trial.suggest_int("n_d", 8, 64)
#     params = {
#             "n_d": n_d,
#             "n_a": n_d,  # n_a는 n_d와 동일하게 설정
#             "n_steps": trial.suggest_int("n_steps", 3, 10),
#             "gamma": trial.suggest_float("gamma", 1.0, 2.0),
#             "n_independent": 2, # 필요하면 3, 4로 늘려본다
#             "n_shared": 2, # 필요하면 3, 4로 늘려본다
#             "lambda_sparse": trial.suggest_float("lambda_sparse", 0.001, 0.01),
#             "optimizer_fn": torch.optim.Adam,
#             "optimizer_params": dict(lr=trial.suggest_float("learning_rate", 0.001, 0.01)),
#             "verbose": 1,
#             "device_name" : "cuda" if torch.cuda.is_available() else "cpu",
#             "seed" : 42
#     }

#     # TabNet 모델 생성
#     model = TabNetRegressor(**params)
    
#     # 모델 학습 및 MAE 계산
#     mae_train, mae_valid = train_model(model, X, y)
#     ### 시각화를 원하면 여기에 넣어주세요. ###
#     print("Optuna 결과..! 💫")
#     print(f"Trial {trial.number}: Train MAE: {mae_train:.4f}, Valid MAE: {mae_valid:.4f}")
    
#     return mae_valid #, model

### 실험 시각화

In [11]:
# # 모델이 학습한 히스토리 확인
# model.history.history.keys()


# fig = plt.figure(figsize=(10, 5))

# # 손실 그래프 그리기
# plt.plot(model.history['loss'])
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Training Loss Over Epochs')
# plt.show()

# # 학습률 그래프 그리기 (현재는 학습률이 고정이라 직선입니다.)
# # model.history에서 learning rate 값을 가져옵니다.
# lr: list[float] = model.history['lr']
# epochs: range = range(1, len(lr) + 1)

# # learning rate 그래프를 그립니다.
# fig = plt.figure(figsize=(10, 5))
# plt.plot(epochs, lr, color='green')
# plt.xlabel("Epoch")
# plt.ylabel("Learning Rate")
# plt.title("Learning Rate over Epochs")
# plt.grid(True)
# plt.show()

In [12]:
# # model.history에서 train MAE와 valid MAE 값을 가져옵니다.
# train_auc: list[float] = model.history['train_mae']
# valid_auc: list[float] = model.history['valid_mae']
# epochs: range = range(1, len(train_auc) + 1)

# # train AUC와 valid AUC 그래프를 그립니다.
# plt.figure(figsize=(10, 6))
# plt.plot(epochs, train_auc, label='Train MAE', color='blue')
# plt.plot(epochs, valid_auc, label='Valid MAE', color='red')
# plt.xlabel("Epoch")
# plt.ylabel("MAE")
# plt.legend()
# plt.title("Train and Valid MAE over Epochs")
# plt.grid(True)
# plt.show()

### Hyperparameter Tuning

In [13]:
# # Optuna 실험 세팅 및 실행
# sampler = optuna.samplers.TPESampler(seed=42)
# study = optuna.create_study(direction="minimize", sampler=sampler)
# study.optimize(objective, n_trials=10)

# # 최적 하이퍼파라미터 출력
# best_params = study.best_params
# print("Best hyperparameters: ", best_params)
# print("Best MAE: ", study.best_value)

In [ ]:
best_params = {
    "n_d": 62,
    "n_a": 62,  # n_a는 n_d와 동일하게 설정
    "n_steps": 8,
    "gamma": 1.2533699284830764,
    "n_independent": 2, # 필요하면 3, 4로 늘려본다.
    "n_shared": 2, # 필요하면 3, 4로 늘려본다.
    "lambda_sparse": 0.009596303461374517,
    "optimizer_fn": torch.optim.Adam,
    "optimizer_params": dict(lr=0.009855066118782934),
    "verbose": 1,
    "device_name" : "cuda" if torch.cuda.is_available() else "cpu",
    "seed" : 42
}
# Optuna로 튜닝한 파라미터로 모델 재학습
best_model = TabNetRegressor(**best_params)
best_model.fit(X_train, y_train, 
          eval_set=[(X_train, y_train),(X_valid, y_valid)], 
          eval_name=["train", "valid"],
          eval_metric=["mae"],
          loss_fn=torch.nn.L1Loss(),
          max_epochs=150, 
          patience=10,
          batch_size=2048,
          drop_last=False,
          warm_start=True  # warm start 활성화
)
print("모델 학습이 완료됐습니다. ⏲")

/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 24422.34528| train_mae: 12627.04499| valid_mae: 12614.26329|  0:02:41s
epoch 1  | loss: 9846.15008| train_mae: 8202.48601| valid_mae: 8206.81213|  0:05:23s
epoch 2  | loss: 8027.71512| train_mae: 7239.30863| valid_mae: 7236.95592|  0:08:18s
epoch 3  | loss: 7099.26364| train_mae: 6205.86109| valid_mae: 6204.70454|  0:11:13s
epoch 4  | loss: 6489.0099| train_mae: 5917.80169| valid_mae: 5921.38333|  0:14:06s
epoch 5  | loss: 6136.78871| train_mae: 5678.81688| valid_mae: 5694.92745|  0:17:04s
epoch 6  | loss: 5914.27421| train_mae: 5378.17398| valid_mae: 5393.93553|  0:20:07s
epoch 7  | loss: 5666.37635| train_mae: 5320.03138| valid_mae: 5346.07817|  0:23:08s
epoch 8  | loss: 5530.00947| train_mae: 5125.90529| valid_mae: 5154.98683|  0:26:10s
epoch 9  | loss: 5406.55442| train_mae: 5379.83802| valid_mae: 5403.17694|  0:29:08s
epoch 10 | loss: 5291.59724| train_mae: 4992.29855| valid_mae: 5024.12225|  0:32:09s
epoch 11 | loss: 5210.04842| train_mae: 4854.43865| valid_mae: 

### Evaluate & Save File

RuntimeError: CUDA error: device-side assert triggered 에러는 주로 GPU에서 데이터 처리 중 발생하는 오류로, 다음과 같은 원인이 있을 수 있습니다:

- 잘못된 입력 데이터 형식: 입력 데이터가 모델이 기대하는 형식과 일치하지 않거나 잘못된 값을 포함할 수 있습니다. 예를 들어, TabNet 모델에 대한 입력 데이터는 float 타입이어야 하며, 정수 인덱스 또는 NaN 값이 없어야 합니다.

- 타겟 변수의 범위 문제: 예측할 때 타겟 변수가 예상 범위를 벗어나는 경우에도 이런 오류가 발생할 수 있습니다.

- 배치 크기 문제: 배치 크기가 너무 크거나 너무 작아서 발생할 수 있습니다.

- CUDA 드라이버 문제: 사용 중인 CUDA 버전이 PyTorch와 호환되지 않거나, 드라이버 업데이트가 필요할 수 있습니다.

**CUDA 런타임 블록 설정**
오류 위치 정확히 확인하기 위해 환경 변수 설정

In [16]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
torch.backends.cuda.matmul.allow_tf32 = True


입력 데이터 확인 -> float타입, NaN값이 없어야 함

In [ ]:
test_data.isnull().sum()

interest_rate에 결측치 확인 -> mean대체

In [ ]:
# # interest_rate 컬럼의 결측값을 평균값으로 대체
test_data["interest_rate"].fillna(test_data["interest_rate"].mean(), inplace=True)

In [22]:
# 1. 원래 데이터 쓰는 사람용 (test_data에서 불필요한 변수 drop: 로그변환한 변수)
X_test = test_data.drop(columns=["index", "log_area_m2", "log_subway_distance", "log_school_distance", "log_park_distance", "log_leader_distance"], inplace=True)
# 2. 로그변환한 데이터 쓰는 사람용 (test_data에서 불필요한 변수 drop: 로그변환하기 전 변수)
#X_test = test_data.drop(columns=["index", "area_m2", "nearest_subway_distance", "nearest_school_distance", "nearest_park_distance", "nearest_leader_distance"], inplace=True)
#X_test = X_test.values.astype(np.float32) # float32 넘파이 배열로 변환
# X_test = test_data[selected_cols]

In [23]:
X_test = test_data.values.astype(np.float32)

In [25]:
# X_test에 대한 예측 수행 후, 예측 결과를 csv 파일로 저장
# 1. 원래 데이터 쓰는 사람용

y_pred = best_model.predict(X_test)
#y_pred = np.expm1(y_pred)
sample_submission["deposit"] = y_pred
sample_submission.to_csv("output.csv", index=False)

# 2. 로그변환한 데이터 쓰는 사람용
#save_csv(best_model, X_test, sample_submission)

In [ ]:
y_pred.mean()

In [ ]:
train_data["deposit"].mean()

In [ ]:
pd.Series(y_pred.flatten()).describe()